In [226]:
import numpy as np
import pandas as pd
import random
import copy

In [227]:
transition_matrix = pd.DataFrame(np.random.random((10,10)))*100
transition_matrix = transition_matrix.astype(int)
transition_matrix

,0,1,2,3,4,5,6,7,8,9
0,57,50,27,52,77,39,43,98,53,7
1,0,21,88,79,64,30,60,63,78,61
2,7,76,24,62,61,9,10,18,61,48
3,46,27,5,99,86,41,18,15,74,53
4,91,94,99,69,53,96,8,14,55,58
5,17,8,7,0,60,86,24,16,76,45
6,87,47,31,84,4,20,45,13,91,94
7,99,53,74,30,94,83,12,87,71,52
8,93,10,88,49,41,95,19,15,64,13
9,24,71,1,82,90,81,16,99,52,45


In [228]:
# initiate chromsome; transitions through 0->10
# chromosome is list 
# fitness test uses list to read through transition matrix;
# product start is row, product end is column; returns cum. 'loss'
# obj is to minimize fitness
# first mutation option will be to swap two numbers in list

In [229]:
class Chromosome():
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness

In [230]:
class GeneSet():
    def __init__(self, products):
        self.products = products
        
    def generate_parent(self):
        return random.shuffle(self.products)

In [231]:
def generate_geneset():
    products = list(np.arange(10))
    return GeneSet(products)

In [232]:
gene_set = generate_geneset()
gene_set.generate_parent()
gene_set.products

[3, 4, 9, 8, 2, 1, 5, 0, 7, 6]

In [233]:
def mutate(parent):
    a, b = parent.index(random.choice(parent)), parent.index(random.choice(parent))
    parent[a], parent[b] = parent[b], parent[a] 
    return parent

In [234]:
mutate(copy.deepcopy(gene_set.products))

[8, 4, 9, 3, 2, 1, 5, 0, 7, 6]

In [235]:
def get_fitness(candidate, transition_matrix):
    score = 0
    for index, product in enumerate(candidate):
        if index < len(candidate)-1:
            cost = transition_matrix.iloc[product, candidate[index+1]]
            transition = product,candidate[index+1]
        else:
            cost = transition_matrix.iloc[product,candidate[0]]
            transition = product, candidate[0]
        score = score + cost
    return score

In [244]:
parent = generate_geneset()
parent.generate_parent()
best_fitness = get_fitness(parent.products, transition_matrix)
best = Chromosome(gene_set.products, best_fitness)
allowed = 1e4
iterations = 0
history = pd.DataFrame([str(best.Genes), best.Fitness, iterations]).T

while True:
    if iterations > allowed:
        break
    iterations += 1
    
    # construct candidate
    genes = mutate(copy.deepcopy(best.Genes))
    fitness = get_fitness(genes, transition_matrix)
    candidate = Chromosome(genes,fitness)
    
    # select candidate or parent
    if candidate.Fitness < best.Fitness:
        best = candidate
        new = pd.DataFrame([str(best.Genes), best.Fitness, iterations]).T
        history = pd.concat([history, new])
    else:
        continue
history.columns = ['Genes', 'Fitness', 'Iteration']
history

,Genes,Fitness,Iteration
0,"[3, 4, 9, 8, 2, 1, 5, 0, 7, 6]",525,0
0,"[6, 4, 9, 8, 2, 1, 5, 0, 7, 3]",471,7
0,"[6, 4, 8, 9, 2, 1, 5, 0, 7, 3]",342,9
0,"[6, 4, 8, 9, 2, 1, 5, 3, 7, 0]",336,15
0,"[6, 4, 8, 9, 2, 7, 5, 3, 1, 0]",244,18
0,"[6, 4, 8, 9, 2, 0, 5, 3, 1, 7]",221,26
0,"[6, 4, 8, 9, 1, 0, 5, 3, 2, 7]",217,90
0,"[6, 4, 8, 9, 0, 1, 5, 3, 2, 7]",211,136
0,"[6, 4, 8, 1, 0, 9, 5, 3, 2, 7]",192,207
